In [1]:
import pandas as pd
import os
import json

pd.set_option('display.max_columns', None)
path = r"C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw"
save_path = r"C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed"

In [2]:
#01 Banco de dados org 
df_org = pd.read_csv(path+"/BDorg/jogos.gz", compression="gzip")
df_org.to_excel(save_path+"/01_games.xlsx", index=False)

In [3]:
#02 Kaggle Adão
df_KG_AD_jogos = pd.read_csv(path+"/kaggle_AD/jogos.csv", sep=",")
df_KG_AD_gols = pd.read_csv(path+"/kaggle_AD/gols.csv", sep=",")
df_KG_AD_estatisticas = pd.read_csv(path+"/kaggle_AD/estatisticas.csv", sep=",")
df_KG_AD_cartoes = pd.read_csv(path+"/kaggle_AD/cartoes.csv", sep=",")
df_KG_AD_jogos.to_excel(save_path+"/02_games.xlsx", index=False)
df_KG_AD_gols.to_excel(save_path+"/02_goals.xlsx", index=False)
df_KG_AD_estatisticas.to_excel(save_path+"/02_stats.xlsx", index=False)
df_KG_AD_cartoes.to_excel(save_path+"/02_cards.xlsx", index=False)

In [4]:
#03 GH Hugo Brendow
df_GH_HB_jogos = pd.read_csv(path+"/GH_HugoBrendow/jogos.csv", sep=";")
df_GH_HB_jogos.to_excel(save_path+"/03_games.xlsx", index=False)

In [5]:
#04 GH Henrique
df_GH_Henr_jogos = pd.read_csv(path+"/GH_Henrique/jogos.csv", sep=",")
df_GH_Henr_gols = pd.read_csv(path+"/GH_Henrique/gols.csv", sep=",") 
df_GH_Henr_cartoes = pd.read_csv(path+"/GH_Henrique/cartoes.csv", sep=",")
df_GH_Henr_jogos.to_excel(save_path+"/04_games.xlsx", index=False)
df_GH_Henr_gols.to_excel(save_path+"/04_goals.xlsx", index=False)
df_GH_Henr_cartoes.to_excel(save_path+"/04_cards.xlsx", index=False)

In [6]:
#05 GH Fabrimatt
df_GH_Fab_jogos = pd.read_excel(path+"/GH_Fabrimatt/jogos_stat.xlsx")
df_GH_Fab_jogos.to_excel(save_path+"/05_games.xlsx", index=False)


In [7]:
#06 GH Avelar
def ler_json(caminho):
    """
    Lê arquivo JSON e entrega em uma estrutura DataFrame.

    Parâmetros:
    - caminho (str): Caminho completo do arquivo .json.
      Exemplo: r'C:\\Users\\...\\arquivo.json'

    Retorna:
    - df (pd.DataFrame): DataFrame com os dados carregados do JSON.
    """
    with open(caminho, 'r', encoding='utf-8') as arquivo:
        dados = json.load(arquivo)
    df = pd.DataFrame(dados)
    return df

def expandir_json(df: pd.DataFrame, prefixo: str) -> pd.DataFrame:
    """
    Expande a coluna 'pontuacao_geral_{prefixo}' de um DataFrame, que contém dicionários ou JSONs,
    em várias colunas separadas, prefixando os nomes com o valor de 'prefixo'.

    Parâmetros:
        df (pd.DataFrame): DataFrame de entrada contendo a coluna com dados em formato dict/JSON.
        prefixo (str): Prefixo que define o nome da coluna a ser expandida e será usado nas novas colunas.

    Retorna:
        pd.DataFrame: Novo DataFrame com as colunas expandidas e renomeadas com o prefixo.
    """

    coluna_json = df[f'pontuacao_geral_{prefixo}']
    colunas_expandida = pd.json_normalize(coluna_json)
    colunas_expandida.columns = [f'{prefixo}_{col}' for col in colunas_expandida.columns]

    return colunas_expandida



avelar_path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw\GH_Avelar'
arquivos_json = sorted([f for f in os.listdir(avelar_path) if f.endswith('.json')])

df_all_camp = pd.DataFrame()
# aplicando a função para cada arquivo JSON que representa o ano do campeonato
for ano_camp_json in arquivos_json:
    relative_path = avelar_path +'\\' + ano_camp_json

    df_json = ler_json(relative_path)
    df_ano_camp = pd.DataFrame()    

    for rodada in range(len(df_json['partidas'])):

        df = pd.DataFrame(df_json['partidas'][rodada])

        pontuacao_mandante = expandir_json(df, 'mandante')
        pontuacao_visitante = expandir_json(df, 'visitante')

        # Juntar tudo em um único DataFrame
        df_rodada = pd.concat([df.drop(['pontuacao_geral_mandante', 'pontuacao_geral_visitante'], axis=1),pontuacao_mandante,pontuacao_visitante], axis=1) 
        df_rodada['rodada'] = df_json['numero'][rodada]

        df_ano_camp = pd.concat([df_ano_camp, df_rodada],axis=0, ignore_index=True)
    df_ano_camp['ano campeonato'] = ano_camp_json.split('.')[0]
    df_all_camp = pd.concat([df_all_camp, df_ano_camp], axis=0, ignore_index=True)

df_all_camp.to_excel(save_path+"/06_games.xlsx", index=False)


In [8]:
#07 GH Arruda
def unir_xlsx_em_df(pasta):
    """
    Lê todos os arquivos .xlsx de uma pasta e os une em um único DataFrame,
    mantendo apenas o cabeçalho do primeiro arquivo.

    Parâmetros:
    - pasta (str): Caminho da pasta com os arquivos .xlsx.

    Retorna:
    - df_final (pd.DataFrame): DataFrame unificado.
    """
    arquivos = sorted([f for f in os.listdir(pasta) if f.endswith(".xlsx")])
    
    if not arquivos:
        print("Nenhum arquivo .xlsx encontrado.")
        return pd.DataFrame()

    lista_dfs = []

    for i, arquivo in enumerate(arquivos):
        caminho_arquivo = os.path.join(pasta, arquivo)

        if i == 0:
            # Primeiro arquivo: lê com cabeçalho
            df = pd.read_excel(caminho_arquivo)
        else:
            # Restante: ignora o cabeçalho
            df = pd.read_excel(caminho_arquivo, header=None, skiprows=1)
            df.columns = lista_dfs[0].columns  # aplica o cabeçalho do primeiro

        lista_dfs.append(df)

    df_final = pd.concat(lista_dfs, ignore_index=True)
    return df_final

## aplicando a funcao para unir os arquivos
df_GH_Ard_jogos = unir_xlsx_em_df(path+"/GH_Arruda/dados/jogos")
df_GH_Ard_jogos.to_excel(save_path+"/07_games.xlsx", index=False)